In [ ]:
import riiideducation
import pickle
import pandas as pd
import numpy as np

In [ ]:
with open('../input/multipledt1/decision_tree.pkl', 'rb') as f:
    dt_list, ques, user = pickle.load(f)

In [ ]:
import riiideducation

#Create the env
env = riiideducation.make_env()

#Create the iterator
iter_test = env.iter_test()

#Iter and predict
for (test_df, sample_prediction_df) in iter_test:
    test_df_q = pd.merge(test_df, ques, how='left',on='content_id')
    test_df_q = pd.merge(test_df_q, user,  how='left', on='user_id')
    test_df_q['q_difficulty'] = test_df_q['q_+_response']/ test_df_q['total_responses_to_q']
    test_df_q['user_stats'] = test_df_q['u_+_response']/ test_df_q['total_user_responses']
    res = np.zeros([len(test_df_q),2])
    for dt in range(len(dt_list)):
        dt_now = dt_list[dt]
        idx = dt_now.predict(test_df_q[['total_responses_to_q', 'q_difficulty', 'total_user_responses', 'user_stats']].fillna(0))
        for j in range(len(test_df_q)):
            res[j,idx[j]] += 1
    test_df['answered_correctly'] = np.argmax(res, axis=1)
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])